---
title: Update the DHIS2 analytics tables
short_title: Update analytics tables
---

When importing climate data into DHIS, you will normally have to wait before you can actually use or access the imported data in your DHIS2, until the next scheduled (typically nightly) run of the analytics resource tables. Or if you want it immediately, this notebook shows how you can manually trigger an update using the python-client. 

## Connect to DHIS2

First, we connect the python-client to the DHIS2 instance we want to import into. In this case we use one of the public access DHIS2 instances that is continuously reset:

In [1]:
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Client configuration
cfg = ClientSettings(
  base_url="https://play.im.dhis2.org/stable-2-42-3-1",
  username="admin",
  password="district")

client = DHIS2Client(settings=cfg)
info = client.get_system_info()

# Check if everything is working.
# You should see your current DHIS2 version info.
print("▶ Current DHIS2 version:", info["version"])

▶ Current DHIS2 version: 2.42.3.1


## Start the analytics job

Here's how to start an analytics job using the python-client: 

In [ ]:
run_analytics_resp = client.post("/api/resourceTables/analytics", params={})
run_analytics_resp

{'httpStatus': 'OK',
 'httpStatusCode': 200,
 'status': 'OK',
 'message': 'Initiated ANALYTICS_TABLE (1765389419122)',
 'response': {'name': 'ANALYTICS_TABLE (1765389419122)',
  'id': 'm0JoKeId1Lw',
  'created': '2025-12-10T17:56:59.122',
  'jobType': 'ANALYTICS_TABLE',
  'jobStatus': 'NOT_STARTED',
  'jobParameters': {'lastYears': 1,
   'skipTableTypes': [],
   'skipPrograms': [],
   'skipResourceTables': False,
   'skipOutliers': False},
  'relativeNotifierEndpoint': '/api/system/tasks/ANALYTICS_TABLE/m0JoKeId1Lw',
  'responseType': 'JobConfigurationWebMessageResponse'}}

## Wait for the job to finish

In the response we use the `relativeNotifierEndpoint` URL to query the progress of the analytics tables, until the `completed` flag returns `True`: 

In [3]:
import time
start_time = time.time()
timeout = 60 * 5 # seconds
logs = []
while True:
    # check the latest logs from the analytics job
    analytics_job_logs = client.get(run_analytics_resp['response']['relativeNotifierEndpoint'])
    # print any new logs
    for log in reversed(analytics_job_logs):
        if log not in logs:
            print(log)
            logs.append(log)
    # check for completed flag in the latest log entry
    if analytics_job_logs[0]['completed'] == True:
        print(f'Analytics job completed in {time.time() - start_time} seconds')
        break
    # check if timed out
    if (time.time() - start_time) >= timeout:
        print(f'Analytics job timed out after {timeout} seconds')
        break
    # wait 1 second
    time.sleep(1)

{'level': 'INFO', 'category': 'ANALYTICS_TABLE', 'message': 'Analytics table update process', 'completed': False, 'dataType': 'PARAMETERS', 'data': {'lastYears': 1, 'skipTableTypes': [], 'skipPrograms': [], 'skipResourceTables': False, 'skipOutliers': False}, 'id': 'C8aS6rX8oQ8', 'time': '2025-12-10T17:56:59.170', 'uid': 'C8aS6rX8oQ8'}
{'level': 'INFO', 'category': 'ANALYTICS_TABLE', 'message': 'Drop SQL views', 'completed': False, 'id': 'C6eW6rX8oO0', 'time': '2025-12-10T17:56:59.172', 'uid': 'C6eW6rX8oO0'}
{'level': 'INFO', 'category': 'ANALYTICS_TABLE', 'message': '0 successful and 0 failed items', 'completed': False, 'id': 'dG0gQ5lJI6u', 'time': '2025-12-10T17:56:59.173', 'uid': 'dG0gQ5lJI6u'}
{'level': 'INFO', 'category': 'ANALYTICS_TABLE', 'message': 'Generating resource tables', 'completed': False, 'id': 'C8eW6rX8oQ4', 'time': '2025-12-10T17:56:59.174', 'uid': 'C8eW6rX8oQ4'}
{'level': 'LOOP', 'category': 'ANALYTICS_TABLE', 'message': '[0/1] Generating resource tables', 'complete